In [11]:
import nbimporter
import functions
reload(functions)
import pandas as pd
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import OrderedDict
import numpy as np
#viz
from datetime import date
from random import randint
import math

from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn
from bokeh.io import output_file, show

from sklearn import ensemble
from sklearn.metrics.pairwise import linear_kernel
from dateutil import parser

from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import arma_order_select_ic
from matplotlib import pyplot

#from bokeh.io import vform

Importing Jupyter notebook from functions.ipynb


In [12]:
##variables
look_back = 2

In [13]:
functions.test()

'test1'

In [14]:
log_file = '/Users/momori/Documents/projects/log_monitoring/muleprep_app_ERROR.csv'
log_file1 = '/Users/momori/Documents/projects/log_monitoring/muleprod_app_aug_7.csv'
log_file2 = '/Users/momori/Documents/projects/log_monitoring/muleprod_app_aug_1.csv'
log_file3 = '/Users/momori/Documents/projects/log_monitoring/muleprep_app_ERROR.csv'
log_file4 = '/Users/momori/Documents/projects/log_monitoring/muleprep_app_ERROR.csv'

df_test = pd.read_csv(log_file)
df_train1 = pd.read_csv(log_file)#1)
df_train2 = pd.read_csv(log_file)#2)

In [15]:
train_df = [df_train1, df_train2]

## Functions

In [16]:
##km: clustering model
##vecetorizer: document vectorizer
##document: document to cluster
def predict_document_cluster(km, vectorizer, document):
    doc = vectorizer.transform([document])
    return km.predict(doc)

def format_df(df):
    r_df = df.dropna(axis=1)
    r_df.loc[:,'_time'] = r_df['_time'].apply(lambda x: format_time(x))
    return r_df

def format_time(string):
    ##may need smarter logic. seems slow
    return parser.parse(string)

##step_time in minutes
##
## returns: dict. key is timestamp, value is dataframe
def split_by_time(df, step_time=30):
    r_dict = {}
    t=str(step_time)+'Min'
    
    #set _time to index
    tmp = df.set_index(1)
    tmp = tmp.groupby(pd.TimeGrouper(freq=t))
    for key, item in tmp:
        try:
            r_dict[str(key)] = tmp.get_group(key)
        except:
            continue
    return r_dict
        
def show_top_words(km, vect, show=False, save=True, out_filename='output'):
    dcluster_words = {}
    if show:
        print("Top terms per cluster:")

    order_centroids = km.cluster_centers_.argsort()[:, ::-1]

    terms = vect.get_feature_names()
    for i in range(k):
        if show:
            print("Cluster %d:" % i)
            for ind in order_centroids[i, :10]:
                print(' %s' % terms[ind])
        if save:
            words = []
            for ind in order_centroids[i, :10]:
                words.append(terms[ind])
            dcluster_words[i] = words
    if save:
        np.save(out_filename, dcluster_words)


In [17]:
#show_top_words(km, vect, save=True)
!ls

Untitled.ipynb               module_test.ipynb
Untitled1.ipynb              new_df_formatted
bollinger.html               new_logs
data_table.html              output.npy
functions.ipynb              splunk log analyzer v2.ipynb
line_bar.html                splunk log analyzer.ipynb
log_analysis_tutorial.ipynb  splunk_log_analyzer_v3.ipynb
log_cleaning.ipynb


In [18]:
d = np.load('output.npy').item()
for k,v1 in d.iteritems():
    print k,v1

0 [u'320bbec0', u'55', u'b667', u'42ff', u'd7c5e8e7d50a', u'f96e', u'29', u'6335', u'account', u'enterpriseaccount']
1 [u'order', u'avs', u'auth', u'payment', u'service', u'errors', u'code', u'failed', u'type', u'invalid']
2 [u'entitlement', u'entitlementofferingrestransformer', u'entitledoffering', u'transformer', u'5266', u'received', u'46', u'connectorwithoutmulesession', u'45', u'02']
3 [u'provided', u'customeraccountnumber', u'invalid', u'moreinfo', u'errors', u'code', u'type', u'202106', u'ent', u'entitledofferings']
4 [u'java', u'org', u'mule', u'extendedschemavalidationfilter', u'runtimeexception', u'wrapper', u'222', u'lang', u'exceptions', u'schemavalidation']
5 [u'00', u'currency', u'usd', u'ns1', u'globalsalestax', u'productclass', u'totalcredit', u'salestaxheader', u'documentid', u'taxtype']
6 [u'licensenumber', u'missing', u'parameter', u'required', u'errors', u'202103', u'code', u'type', u'ent', u'entitledofferings']
7 [u'multiple', u'schemavalidationfilter', u'rejected'

In [ ]:
logs_train1 = df_train1['_raw'].tolist()
logs_train2 = df_train2['_raw'].tolist()
logs_test = df_test['_raw'].tolist()

In [ ]:
##Fit a vectorizer
vect = TfidfVectorizer(max_df = 0.5, min_df = 2, stop_words='english')

#v is base vectorizer, can be used to transform other logs
v = vect.fit(logs_train1)

#x is the tfidf sparse matrix created for this set of logs
x1 = v.transform(logs_train1)
x2 = v.transform(logs_train2)

In [ ]:
##create kmeans model and fit
k=20
km = MiniBatchKMeans(n_clusters=k, init='k-means++', n_init=1,
                         init_size=1000, batch_size=1000)
km.fit(x1)

In [ ]:
##Let's use cluster 6 as it shows '500' as one of the top words
show_top_words(km, vect, show=True)

In [ ]:
#for i in range(20):
#    print i, len(df_predict1[df_predict1[2]==i]), len(df_predict2[df_predict2[2]==i])

In [ ]:
##get predictions for a subset of the logs and create df with _raw, _time, and label
formatted_df = format_df(df_train1)

logs = formatted_df['_raw'].tolist()
times = formatted_df['_time'].tolist()
test_x1 = v.transform(logs)

labels = km.predict(test_x1)
lst = zip(logs, times, labels.tolist())
df_predict1 = pd.DataFrame(lst)
df_predict1.head()

##do same for second train set
formatted_df = format_df(df_train2)

logs = formatted_df['_raw'].tolist()
times = formatted_df['_time'].tolist()
test_x2 = v.transform(logs)

labels = km.predict(test_x2)
lst = zip(logs, times, labels.tolist())
df_predict2 = pd.DataFrame(lst)
df_predict2.head()


In [ ]:
df_predict1.shape

In [ ]:
##for each cluster number, create a dt
df_clusters1, df_clusters2 = None


In [ ]:
cluster_num = 8
f_df1 = df_predict1[df_predict1[2]==cluster_num]
f_df2 = df_predict2[df_predict2[2]==cluster_num]
#f_df1 = df_predict1
#f_df2 = df_predict2
f_df1.head()

In [ ]:
check_logs = f_df1[0].tolist()

In [ ]:
check_logs

In [ ]:
##Create dictionary of dataframes, split by time
d1 = OrderedDict(sorted(split_by_time(f_df1).items()))
d2 = OrderedDict(sorted(split_by_time(f_df2).items()))
#d1 = OrderedDict(sorted(split_by_time(df_predict1).items()))
#d2 = OrderedDict(sorted(split_by_time(df_predict2).items()))


In [ ]:
#create counts for training
counts1 = []
counts2 = []
for k, val in d1.iteritems():
    counts1.append(val.shape[0])
for k, val in d2.iteritems():
    counts2.append(val.shape[0])

In [ ]:
len(counts1), len(counts2)

In [ ]:
counts1[:10], counts2[:10]

In [ ]:
v = vect.fit(logs_train1)
v.transform(logs_test)

In [ ]:
##Let's fit the 'future' data and pull out cluster_num
formatted_df = format_df(df_test)
logs_test = formatted_df['_raw'].tolist()
times = formatted_df['_time'].tolist()
test_x1 = v.transform(logs_test)

labels = km.predict(test_x1)
lst = zip(logs, times, labels.tolist())
test_data = pd.DataFrame(lst)
test_data.head()

In [ ]:
test_data = test_data[test_data[2]==cluster_num]

#test_data = test_data
test_d = OrderedDict(sorted(split_by_time(test_data).items()))
test_counts = []
for k, val in test_d.iteritems():
    test_counts.append(val.shape[0])


In [ ]:
test_data.head()

In [ ]:
len(test_counts)

In [ ]:
test_counts[:10]

In [ ]:
#zip the data
counts1 = [float(i) for i in counts1]
counts2 = [float(i) for i in counts2]

train = zip(counts1, counts2)
train = [i for i in train if math.fabs(i[1]-i[0] < 100)]
print len(train)

## ARIMA prediction

In [ ]:
##make predictions by fitting ARIMA model into the lists
prediction_upperbound = []
prediction_lowerbound = []
prediction_value = []

for t in train:
    rgr = ARIMA(t, order=(0,0,0))
    fit = rgr.fit(disp=0)
    forecast =  fit.forecast()
    prediction_upperbound.append(forecast[2][0][1])
    prediction_lowerbound.append(forecast[2][0][0])
    prediction_value.append(forecast[0])

#Turn into numpy array for computation
prediction_upperbound = np.asarray(prediction_upperbound)
prediction_lowerbound = np.asarray(prediction_lowerbound)
prediction_value = np.asarray(prediction_value)

## Vis

In [ ]:
import numpy as np

from bokeh.plotting import figure, show, output_file

# Define Bollinger Bands.
upperband = prediction_upperbound
lowerband = prediction_lowerbound
x_data = np.arange(1, 101)

length = min(len(prediction_upperbound), len(prediction_lowerbound))
x_data = np.arange(1, length)
# Bollinger shading glyph:
band_x = np.append(x_data, x_data[::-1])
band_y = np.append(lowerband, upperband[::-1])

output_file('bollinger.html', title='Bollinger bands (file)')


p = figure(x_axis_type='datetime', title="")

# add a line renderer
p.line([i for i in range(1,length)], counts2, line_width=2)
p.grid.grid_line_alpha = 0.4
p.x_range.range_padding = 0
p.plot_height = 600
p.plot_width = 800

p.patch(band_x, band_y, color='#7570B3', fill_alpha=0.2)

show(p)

In [ ]:
f = '/Users/momori/PycharmProjects/AIMonitoring/data/qq.csv'
dd = pd.read_csv(f)

In [ ]:
dd

# Test

In [ ]:
##get predictions for a subset of the logs and create df with _raw, _time, and label
test_df = format_df(df[0:10000])

logs = test_df['_raw'].tolist()
times = test_df['_time'].tolist()
test_x = vect.transform(logs)

labels = km.predict(test_x)
lst = zip(logs,times,labels.tolist())
df_predict = pd.DataFrame(lst)
df_predict

In [ ]:
##Create dictionary of dataframes, split by time
d = split_by_time(df_predict)
for k,v in d.iteritems():
    print k, ':::', type(v)
    

In [ ]:
d['2017-08-19 13:00:00-07:00']

In [ ]:
##Cluster the interval logs by using km.  
#Then get the count for predicting outcomes.

#test out on d
counts = []
for k, v in d.iteritems():
    print k
    counts.append(v.shape[0])



In [ ]:
counts

In [ ]:
##train gradient boosted trees for regression
params = {'n_estimators': 10, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
clf = ensemble.GradientBoostingRegressor(**params)
clf

## Use an ARIMA Model to predict

In [ ]:
floats = [float(i) for i in counts]
floats

In [ ]:
rgr = ARIMA(floats[:len(floats)-1], order=(0,0,0))

In [ ]:
fit = rgr.fit(disp=0)

In [ ]:
fit.forecast()

## test area

In [ ]:
import numpy as np

from bokeh.plotting import figure, show, output_file

# Define Bollinger Bands.
upperband = np.random.random_integers(100, 150, size=100)
lowerband = upperband - 100
x_data = np.arange(1, 101)

# Bollinger shading glyph:
band_x = np.append(x_data, x_data[::-1])
band_y = np.append(lowerband, upperband[::-1])

output_file('bollinger.html', title='Bollinger bands (file)')


p = figure(x_axis_type='datetime', title="Bollinger Bands")

# add a line renderer
p.line([0, 20, 30, 40, 50], [80, 130, 135, 90, 90], line_width=4)
p.grid.grid_line_alpha = 0.4
p.x_range.range_padding = 0
p.plot_height = 600
p.plot_width = 800

p.patch(band_x, band_y, color='#7570B3', fill_alpha=0.2)

show(p)

In [ ]:
# from bokeh.plotting import figure, output_file, show
# from bokeh.models.ranges import Range1d
# import numpy


# output_file("line_bar.html")

# p = figure(plot_width=400, plot_height=400)

# # add a line renderer
# p.line([1, 2, 3, 4, 5], [6, 7, 6, 4, 5], line_width=2)

# # setting bar values
# h = numpy.array([2, 8, 5, 10, 7])

# # Correcting the bottom position of the bars to be on the 0 line.
# adj_h = h/2

# # add bar renderer
# p.rect(x=[1, 2, 3, 4, 5], y=adj_h, width=0.4, height=h, color="#CAB2D6")

# # Setting the y  axis range   
# p.y_range = Range1d(0, 12)

# #p.title = "Line and Bar"

# show(p)

In [ ]:
# from bokeh.plotting import figure, output_file, show
# from bokeh.models.ranges import Range1d
# import numpy


# output_file("line_bar.html")
# p = figure(plot_width=400, plot_height=400)

# p.line([1, 2, 3, 4, 5], [6, 7, 6, 4, 5], line_width=2)


# show(p)